In [1]:
# import torch
import json
# torch.cuda.empty_cache()

## Create Prompt

In [20]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

In [21]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [23]:
# A simple prompt to extract information from "student_description" in a JSON format.
sys_message = '''
Please extract the following information from the given text and return it as a JSON object:
name: as a list
major: as a list
school: as a list
grades: as a list
club: as a list
'''

def get_prompt(sys_message: str, query: str):
    # note, don't "</s>" to the end
    return f'<s> [INST] {sys_message} [/INST]\nUser: {query}\nAssistant: '

def get_prompt(sys_message: str, query: str):
    # note, don't "</s>" to the end
    return f'<|im_start|>system\n{sys_message}<|im_end|>\n<|im_start|>user\n{query}<|im_end|>\n<|im_start|>assistant'

In [24]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
print(models)
model = models.data[0].id

SyncPage[Model](data=[Model(id='meta-llama/Llama-3.2-3B-Instruct', created=1730265949, object='model', owned_by='vllm', root='meta-llama/Llama-3.2-3B-Instruct', parent=None, max_model_len=4096, permission=[{'id': 'modelperm-e8737bf0c0694ed4806d63c334cf5966', 'object': 'model_permission', 'created': 1730265949, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')


In [31]:
full_prompts = [get_prompt(sys_message, i) for i in [student_1_description, student_2_description]]

In [ ]:
# Generating response back from gpt-3.5-turbo
stream = False
openai_response = client.completions.create(
    model=model,
    prompt=full_prompts,
    echo=False,
    stop= ['\nUser:'],
    max_tokens=512,
    temperature=0,
    frequency_penalty= 0,
    presence_penalty= 0,
    n=1,
    best_of=1,
    stream=stream,
    top_p=1,
    logprobs=None)

json_response = json.loads(openai_response.choices[0].text)
json_response

In [28]:
from pydantic import BaseModel, Field
from typing import Optional, List

# define function structure in pydantic

class ExtractInsightsParams(BaseModel):
    person_name: List[str] = Field(..., description="List of names of the people, e.g. Christoph, person y")
    major: List[str] = Field(..., description="List of majors")
    school: List[str] = Field(..., description="List of schools")
    grades: List[str] = Field(..., description="List of grades")
    club: List[str] = Field(..., description="List of clubs")

ExtractInsightsParams.model_json_schema()

{'properties': {'person_name': {'description': 'List of names of the people, e.g. Christoph, person y',
   'items': {'type': 'string'},
   'title': 'Person Name',
   'type': 'array'},
  'major': {'description': 'List of majors',
   'items': {'type': 'string'},
   'title': 'Major',
   'type': 'array'},
  'school': {'description': 'List of schools',
   'items': {'type': 'string'},
   'title': 'School',
   'type': 'array'},
  'grades': {'description': 'List of grades',
   'items': {'type': 'string'},
   'title': 'Grades',
   'type': 'array'},
  'club': {'description': 'List of clubs',
   'items': {'type': 'string'},
   'title': 'Club',
   'type': 'array'}},
 'required': ['person_name', 'major', 'school', 'grades', 'club'],
 'title': 'ExtractInsightsParams',
 'type': 'object'}

In [32]:
stream = False
openai_response = client.completions.create(
    model=model,
    prompt=full_prompts,
    echo=False,
    stop= ['\nUser:'],
    max_tokens=512,
    temperature=0,
    frequency_penalty= 0,
    presence_penalty= 0,
    n=1,
    best_of=1,
    stream=stream,
    top_p=1,
    logprobs=None,
    extra_body={
        "guided_json": ExtractInsightsParams.model_json_schema()
    })

# json_response = json.loads(openai_response.choices[0].text)
# json_response

In [34]:
for ch in openai_response.choices:
    json_response = json.loads(ch.text)
    print(json_response)

{'major': ['Computer Science'], 'school': ['Stanford University'], 'grades': ['3.8'], 'club': ['Robotics Club'], 'person_name': ['David Nguyen']}
{'major': ['Computer Science'], 'school': ['University of Michigan'], 'grades': ['3.7'], 'club': ['Chess Club', 'South Asian Student Association'], 'person_name': ['Ravi Patel']}
